In [1]:
DOC_DIRECTORY = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant - Copy\data\processed\all_split.pkl'
# Sample_DIRECTORY = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant\data\processed\sys_sample.pkl'


# Specify the directory where you want to save the vector database
PERSIST_DIRECTORY = r'D:\Projects\RAG-Powered-Arabic-AI-Assistant - Copy\data\processed\vector_db'




In [2]:
import os
import pickle
from sentence_transformers import SentenceTransformer
from langchain_chroma import Chroma
from langchain.docstore.document import Document
from tqdm import tqdm  # Import tqdm for the progress bar


import markdown
from langchain.prompts import PromptTemplate  ,FewShotPromptTemplate ,ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough

from langchain.chains import SequentialChain
from langchain.schema import  StrOutputParser

google_api_key = os.getenv('GOOGLE_API_KEY')
AI2_API_KEY = os.getenv('AI2_API_KEY')

c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


# Functions

In [3]:
def format_docs(docs):
    return "\n\n". join(doc.page_content for doc in docs)

In [4]:
# Initialize the model 
model = SentenceTransformer('all-distilroberta-v1')

class embedding:
    def __init__(self):
        self.model = model
    def embed_documents(self , docs):
        embeddings = self.model.encode(docs)
        return embeddings.tolist()
    
    def embed_query(self , query):
        return self.model.encode(query).tolist()
    
embed_model = embedding()

c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Load data and data base

In [5]:
with open(DOC_DIRECTORY, 'rb') as f:
    chunks = pickle.load(f)

In [6]:
len(chunks)

45500

In [7]:
chunks[101]

Document(metadata={'source': 'Tech\\0101.txt'}, page_content='أعلنت شركة إكستريم نتووركس، أمس، عن تعيين أوسكار رودريغز في منصب الرئيس التنفيذي والمدير العام للشركة . وينضم رودريغز إلى إكستريم نتووركس وفي رصيده ما يزيد على 20 عاماً من الخبرة في مجال المبيعات والتسويق .صرّح غوردون ستيت، رئيس مجلس إدارة الشركة قائلاً: ينضم رودريغز إلى إكستريم نتووركس حاملاً معه إرثاً غنياً والخبرة المطلوبة في مجال تسويق التقنيات فضلاً عن الصفات القيادية المناسبة لإدارة الشركة، وتابع ستيت قائلاً: ونحن بحاجة لشخص بحجم خبرته وعلاقاته لكي يساعدنا على تطوير الشركة وتوسيع آفاقها وتحريك عجلة نموّها .وصرّح رودريغز قائلاً: جُلّ ما لفتني في إكستريم نتووركس العمل الدؤوب الذي أنجزته الشركة في الأشهر الأخيرة، حيث واظبت على تطوير فعاليتها وتجنيد طاقاتها بما أعاد إحياء المنافسة في السوق وجعلها رائدة في مجالها .')

In [8]:
cat_name = 'Tech'
file_names = [f"{cat_name}\\{str(i).zfill(4)}.txt" for i in range(6500)]

In [9]:
# Later, you can load the database like this:
loaded_vector_data = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    embedding_function = embed_model
)
reteiever = loaded_vector_data.as_retriever(search_type = 'similarity' , search_kwargs= {'k' : 10 ,'filter':{ 'source': {'$in': file_names}}})


In [10]:
q = chunks[-1].page_content
reteiever.invoke(q)

[Document(metadata={'source': 'Religion\\6499.txt'}, page_content='وجود قدر من الغيرة قد يحرك الماء الراكد في العلاقة بين الزوجين، وعلى مقدار هذه الغيرة يتحدد شكل العلاقة، واختفاؤها قد يعني أن الحب والإحساس بالمسؤولية معدومان، وتعتبر الغيرة من توابل الحب إذا كانت بالمقدار المعتدل، وهي مطلوبة شرعاً وعرفاً، إلا أن منها ما يقتل صاحبه ويدمر حياته أو يتحول إلى قنبلة موقوتة قد تنفجر في أية لحظة فتدمر الأسرة كلها .والفارق بين الغيرة الملتهبة والشك خيط رفيع، يفشل بعض الأزواج والزوجات في التفرقة بينهما، ويجب ألا تفسد تلك الغيرة حياة صاحبها ومن حوله، وفي بعض الأحيان تكون نتيجتها نهاية مأساوية، ويبدأ الندم في وقت لا يفيد فيه الندم .الدكتور محمد نبيل غنايم، أستاذ الشريعة الإسلامية بكلية دار العلوم في جامعة القاهرة، يؤكد أن الغيرة طبيعة جبل عليها الإنسان السوي الذي كرمه ربه وفضله، وقد أعلى شأنها وقدرها الإسلام، وتظهر هذه الصفة والغريزة الإنسانية أكثر ما تظهر بين الزوجين، لأنها يشترك فيها الرجال والنساء، بل قد تكون أكثر وأشد لدى المرأة .والغيرة في موضعها والاعتدال فيها بالنسبة للرجال والنساء من جملة

# RAG

In [11]:
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    google_api_key=google_api_key
    )


### template

In [22]:
template = """
    انت مساعد ذكي للاشخاص الناطقين باللغة العربيه تساعدهم في معرفة اخر الاخبار في مجالات مثل التكنولوجيا والسياسة والرياضة من الملفات المتاح لك الاطلاع عليها ,وفي نهاية اجابتك اشكر المستخدم واسئله هل لديك اسئلة اخري 
    
knowledge you know:
{context}

Question: {question}

ماذا تفعل إذا لم تكن الإجابة مدرجة في السؤال أو السياق:
1. أخبر المستخدم أنك ليس لديك معلومات كافية للاجابة علي سواله
3. اسأل المستخدم إذا كان لديه المزيد من الأسئلة ليطرحها.
4. لا تذكر أي شيء عن السياق.





answer : 


"""

In [23]:
custom_rag_prompt=ChatPromptTemplate.from_template(template)
custom_rag_prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\n    انت مساعد ذكي للاشخاص الناطقين باللغة العربيه تساعدهم في معرفة اخر الاخبار في مجالات مثل التكنولوجيا والسياسة والرياضة من الملفات المتاح لك الاطلاع عليها ,وفي نهاية اجابتك اشكر المستخدم واسئله هل لديك اسئلة اخري \n    \nknowledge you know:\n{context}\n\nQuestion: {question}\n\nماذا تفعل إذا لم تكن الإجابة مدرجة في السؤال أو السياق:\n1. أخبر المستخدم أنك ليس لديك معلومات كافية للاجابة علي سواله\n3. اسأل المستخدم إذا كان لديه المزيد من الأسئلة ليطرحها.\n4. لا تذكر أي شيء عن السياق.\n\n\n\n\n\nanswer : \n\n\n'))])

In [24]:
rag_chain=(
    {'context':reteiever 
     ,'question' :RunnablePassthrough()}
     |custom_rag_prompt
     |llm
     |StrOutputParser()

)

In [25]:
chunks[45000]

Document(metadata={'source': 'Religion\\6000.txt'}, page_content='د. عارف الشيخ قال لي: مما تعلمته من مشايخنا -رحمهم الله تعالى- أن الأخت الشقيقة والأخت لأب والأخت لأم من أصحاب الفروض، ولهن نصيب مقدر في الشرع.لكن مع ذلك سمعت من المشايخ أن الأقرب يحجب الأبعد، فإذا وجد الأخ الشقيق لم يرث الأخ لأب، أو وجد الابن لم يرث الأخ. ومع ذلك فإن جارنا توفي وترك أختاً شقيقة وأختاً لأب، وأختاً لأم، فوزعت المحكمة تركة المتوفى بينهن كالآتي: أعطت للشقيقة النصف، وللأخت للأب السدس، وللأخت للأم السدس أيضاً، وبقي سدس فردت المحكمة وقالت: هو للحكومة (بيت المال) أو يرد على أصحاب الفروض. يتساءل صاحبنا: كيف تساوت الأخت لأب والأخت لأم مع الأخت الشقيقة مع العلم بأن الشقيقة أقوى فتحجب كما قال المشايخ؟ أقول: إن الله تعالى يقول: «يستفتونك قل الله يفتيكم في الكلالة إن امرؤ هلك ليس له ولد وله أخت فلها نصف ما ترك، وهو يرثها إن لم يكن لها ولد، فإن كانتا اثنتين فلهما الثلثان مما ترك، وإن كانوا إخوة رجالاً ونساء فللذكر مثل حظ الأنثيين» (الآية 176 من سورة النساء). وبما أن الورثة يقسمون إلى أصحاب الفروض وإلى الورثة بالتعصيب،

In [38]:
cat_name = 'Sports'
file_names = [f"{cat_name}\\{str(i).zfill(4)}.txt" for i in range(6500)]

reteiever = loaded_vector_data.as_retriever(search_type = 'similarity' , search_kwargs= {'k' : 15 ,'filter':{ 'source': {'$in': file_names}}})


In [39]:
q = 'اخبار فريق ريال مدريد'
reteiever.invoke(q)

[Document(metadata={'source': 'Sports\\2416.txt'}, page_content='تجربة أفاق'),
 Document(metadata={'source': 'Sports\\4529.txt'}, page_content='قام الملاكم الفلبيني ماني باكياو بزيارة حديقة حيوانات بريزبين بأستراليا، وذلك على هامش استعداده لنزال الأسترالي جيف هورن في يوليو/ تموز المقبل.'),
 Document(metadata={'source': 'Sports\\5277.txt'}, page_content='ما زالت نجمة التنس الهندية سانيا ميرزا تلقى الاستقبالات الحارة في بلادها بعد فوزها مع مواطنها بوباتي بلقب الزوجي المختلط في بطولة استراليا المفتوحة.'),
 Document(metadata={'source': 'Sports\\6354.txt'}, page_content='حل نادي انتر ميلان الايطالي ومدربه خوزيه ضيوفا على أكاديمية كوارا لكرة القدم في نيجيريا والتقى فريق ايلورين على ملعبه وذلك ضمن جولة الانتر الصيفية استعدادا للموسم المقبل. (أ.ب)'),
 Document(metadata={'source': 'Sports\\2774.txt'}, page_content='شاركت نجمة التنس ماريا شارابوفا في حملة لجمع التبرعات في مدينة هواهن التايلندية والتقت برجال دين على هامش مباراة استعراضية أمام الأمريكية فينوس ويليامز وقد فازت النجمة الروسية بالمبا

In [40]:
ans = rag_chain.invoke(q)

In [41]:
print(ans)

ليس لدي معلومات كافية للإجابة على سؤالك.

هل لديك أسئلة أخرى؟


In [ ]:
print(chunks[21])

page_content='دبي - "الخليج":أعلنت شركة "أيسر" عن طرح سلسلة من الحواسيب المحمولة مصممة خصيصاً لتلبية احتياجات المستخدمين من المهنيين العاملين في الشركات الصغيرة والمتوسطة أو في المكاتب الصغيرة والمكاتب المنزلية، ممن يتطلعون إلى التمتع بإمكانيات التنقل السهل والإنتاجية العالية والكفاءة .تضمّ السلسلة الجديدة المسمّاة "إكستينسا 15" (Extensa 15) جهازين هما EX2510 و EX2509مجهزان بشاشة ذات دقة حادة قياسها 6 .15 بوصة ويعملان على نظام التشغيل "ويندوز 1 .8" . ويسمح المعالج القوّي "إنتل كور" Intel( Core من الجيل الرابع في الجهاز EX2510 والمعالج "سيليرون/بنتيوم كواد كور" من "إنتل" في الجهاز EX2509 بأداء مهام متعددة متزامنة بسلاسة وإجراء عمليات بحث مبسّطة سواء عند الاتصال بالإنترنت أو عند عدم اتصال .وعملت "أيسر" على ضبط كل التفاصيل في هذين الجهازين لضمان أقصى قدر من القدرة على التنقل، فزوّدتهما ببطارية تتمتع بواحدة من أطول فترات العمل في فئتها؛ والتي تصل إلى 7 ساعات . وعلاوة على ذلك، تضمّ سلسلة "إكستينسا 15" واحداً من أنحل النماذج في فئتها، فضلاً عن تصميم يسهل إمساكه وحمله، ما يساعد المستخدم على س

In [132]:
for chunk in rag_chain.stream('ماذا أتاحت الإدارة العامة للإقامة وشؤون الأجانب في دبي، بالتعاون مع شركة «إماراتك»،'):
    print(chunk , end="" , flush=True)

أتاحت الإدارة العامة للإقامة وشؤون الأجانب في دبي، بالتعاون مع شركة «إماراتك»، فرصة التسجيل المجاني للمواطنين والمقيمين في البوابة الذكية.

هل لديك المزيد من الأسئلة؟

In [ ]:
reteiever.invoke('ماذا أتاحت الإدارة العامة للإقامة وشؤون الأجانب في دبي، بالتعاون مع شركة «إماراتك»،')

[Document(metadata={'source': 'D:\\Projects\\RAG-Powered-Arabic-AI-Assistant\\data\\raw\\Religion\\0984.txt'}, page_content='أكد الدكتور أحمد دويدار أستاذ التاريخ والحضارة الإسلامية وإمام المركز الإسلامي في منهاتن بنيويورك أن الإعلام الغربي يتخذ موقفا عدائيا من الإسلام والمسلمين وكل ما هو إسلامي، ويروج الأكاذيب بناء على مفاهيم موروثة ومعلومات مغلوطة بعيدا عن الحقيقة والواقع، ونحن نسعى إلى التفاهم والالتقاء مع الآخر حول المبادئ الإنسانية والعيش بسلام من خلال الحوار الذي يدعو إليه ديننا الحنيف، وأن الحوار لا يعني أن يغيّر أحد الطرفين عقيدته. وقال الدكتور دويدار: إن الخلاف العقائدي مع الأمريكيين وحداثة عهد المسلمين بأمريكا وراء عدم قيام لوبي إسلامي مؤثر على خلاف اللوبي الصهيوني المتوغل في الإعلام والمتحكم في الاقتصاد والمشارك في صنع القرار السياسي. ودعا د. أحمد دويدار المسلمين لتحسين سلوكياتهم وتصرفاتهم ومعاملاتهم خاصة مع غير المسلمين والتعامل بما يتوافق مع تعاليم الدين الحنيف وسماحته لأن ذلك في حد ذاته دعوة مؤثرة لنشر الإسلام وتصحيح للمفاهيم المغلوطة الظالمة والموروثة في نفوس الغربيين. و

In [133]:
from IPython.display import display , HTML

In [136]:
html=  markdown.markdown( ans)

display(HTML(html))

# AI2

In [83]:
from langchain_ai21 import ChatAI21

# Replace with your actual API key and desired model
chat_ai2 = ChatAI21(
    api_key=AI2_API_KEY,
    model="jamba-1.5-mini"
)


In [75]:
# help(ChatAI21)

In [80]:
prompt = 'من فضلك، اذكر بالتفصيل أهم الأكلات الشعبية في مصر.'
response = model.invoke(prompt)


In [82]:
response.content

'بالتأكيد، مصر تتمتع بثراء ثقاف'

In [28]:
rag_chain_AI2=(
    {'context':reteiever 
     ,'question' :RunnablePassthrough()}
     |custom_rag_prompt
     |chat_ai2
     |StrOutputParser()

)
ans = rag_chain_AI2.invoke('ما الحدث الذي ياخذ مكانا ف الشارقة؟')
print(ans)


اجتماع الأمانة العامة لمراكز الوثائق


In [36]:
print(chunks[50])

page_content='أتاحت الإدارة العامة للإقامة وشؤون الأجانب في دبي، بالتعاون مع شركة «إماراتك»، فرصة التسجيل المجاني للمواطنين والمقيمين في البوابة الذكية، عن طريق محطة تسجيل في منصتها في معرض جيتكس في مركز دبي التجاري العالمي 2015، الذي انتهت فعالياته يوم الخميس. ولاقت منصة إقامة دبي إقبالا كبيرا من الجمهور حيث تعرفوا الى أحدث الخدمات في مجال التكنولوجيا والخدمات الذكية. وقد سجل في البوابة الذكية خلال المعرض ما يفوق 500 شخص. وتتيح البوابة للمسافرين المسجلين استخدام جواز السفر، أو الهوية الوطنية، أو بطاقة بوابة الإمارات، وحتى الهاتف الذكي، كطرق مختلفة لاستخدام البوابة الذكية. كما زار الفريق سيف عبدالله الشعفار وكيل وزارة الداخلية منصة إقامة دبي في معرض جيتكس 2015 يرافقه وفد من كبار الضباط من وزارة الداخلية وكان في استقباله العميد حسين إبراهيم، مساعد المدير العام لقطاع الدعم المؤسسي في الإدارة العامة للإقامة وشؤون الأجانب في دبي والرائد خالد بن مدية نائب مساعد المدير للخدمات الذكية، كما زار المنصة حسين لوتاه مدير عام بلدية دبي ووفد مرافق. وزار العديد من المسؤولين منصة إقامة دبي واطلعوا على

In [64]:
rag_chain_AI2.invoke('ماذا أتاحت الإدارة العامة للإقامة وشؤون الأجانب في دبي، بالتعاون مع شركة «إماراتك»،')

'أتاحت الإدارة العامة للإقامة وشؤون'

In [39]:

for chunk in rag_chain_AI2.stream('ماذا أتاحت الإدارة العامة للإقامة وشؤون الأجانب في دبي، بالتعاون مع شركة «إماراتك»،'):
    print(chunk , end="" , flush=True)


أتاحت الإدارة العامة للإقامة وشؤون

In [37]:
reteiever.invoke("ماذا أتاحت الإدارة العامة للإقامة وشؤون الأجانب في دبي، بالتعاون مع شركة «إماراتك»،")

[Document(metadata={'source': 'D:\\Projects\\RAG-Powered-Arabic-AI-Assistant\\data\\raw\\Culture\\5080.txt'}, page_content='الكويت - الحسيني البجلاتي: فقدت الكويت أمس الأول أحد أعمدة الحركة الثقافية والأدبية الكويتية والخليجية ، وهو الكاتب المسرحي والأديب والإعلامي سليمان داود الحزامي عن عمر ناهز 70 عاماً بعد صراع مع المرض.وكان الفقيد هو الرئيس الأسبق لرابطة الأدباء الكويتية، ورئيس اللجنة الثقافية بها وعضو مسرح الخليج العربي ورئيس لتحرير مجلة البيان الكويتية، ومستشار تحرير سلسلة إبداعات عالمية الصادرة عن المجلس الوطني للثقافة والفنون والآداب.ونعى وزير الإعلام ووزير الدولة لشؤون الشباب الشيخ سلمان الحمود الفقيد، واصفاً إياه بأنه أحد فرسان وأعمدة ورواد وأعلام الثقافة الكويتية الذي يمثل رحيله خسارة كبيرة للأسرة الثقافية والإعلامية الكويتية والخليجية والعربية على حد سواء، لما كان يتمتع به من دماثة الخلق وإبداعات فكرية وثقافية وإعلامية ستظل نافذة مضيئة في تاريخ الثقافة والإعلام الكويتي.ولد الحزامي في حي شرق بالكويت عام 1945 والتحق بمدرسة المرقاب حتى عام 1952 وتخرج في معهد المعلمين مدرساً لم

In [86]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load model and tokenizer from Hugging Face
model_name = "facebook/bart-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)



c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\.cache\huggingface\hub\models--facebook--bart-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokeniz

ما الحدث الذي ياخذ مكانا في الشارقة؟


In [88]:

def generate_response(prompt):
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(inputs, max_length=500)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

response = generate_response('مرحبا ')
print(response)

مرحبا 


In [115]:
# from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm

llm = GooglePalm(
    model='genini-pro',
    api_key=google_api_key,
    temperature=0.1,
)




In [118]:
llm_result = llm('مرحبا')


c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:151: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


IndexError: list index out of range

In [108]:
llm_result

LLMResult(generations=[[], [], [], [], []], llm_output=None, run=None)

In [117]:
palm=(
    {'context':reteiever 
     ,'question' :RunnablePassthrough()}
     |custom_rag_prompt
     |llm
     |StrOutputParser()

)
ans = palm.invoke('ما الحدث الذي ياخذ مكانا ف الشارقة؟')
print(ans)


Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InvalidArgument: 400 The requested language is not supported by models/text-bison-001.
Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InvalidArgument: 400 The requested language is not supported by models/text-bison-001.
Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised InvalidArgument: 400 The requested language is not supported by models/text-bison-001.
Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 8.0 seconds as it raised InvalidArgument: 400 The requested language is not supported by models/text-bison-001.
Retrying langchain_community.llms.google_palm.completion_with_retry.<locals>._completion_with_retry in 10.0 seconds as it raised InvalidArgu

InvalidArgument: 400 The requested language is not supported by models/text-bison-001